In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt

# Setup n by n grid
# Input: n = number of grid cell
def field(n):
    # Grid Center Location
    center = np.empty(n)
    edge = np.empty(n+1)
    edge[0] = 0
    for i in range(n):
        center[i] = (2*i+1)*1/n*1/2
        edge[i+1] = (i+1)*1/n
    return center, edge

# Output grid center of current position
def grid_check(center, edge, xi):
    x = xi[0]
    y = xi[1]
    n = len(center)
    for i in range(n):
        minedgex = edge[i]
        maxedgex = edge[i+1]
        for j in range(n):
            minedgey = edge[j]
            maxedgey = edge[j+1]
            if minedgex <= x and x <= maxedgex and minedgey <= y and y <= maxedgey:
                grid_center = np.array([[center[i]], [center[j]]])
    return grid_center

In [2]:
# Wrap2Pi
def wrap2pi(x):
    return np.where(np.abs(x) >= np.pi, (x+np.pi)%(2*np.pi)-np.pi, x)

In [3]:
n = 21
x = np.linspace(0, 1, n)
y = np.linspace(0, 1, n)
X,Y = np.meshgrid(x, y)

# Define Initial, Final position of the aggressor
def position(i_x):
    return np.array([x[i_x[0]], y[i_x[1]]])

i_x0 = (int(0.2*n), int(0.9*n))
i_xf = (int(0.9*n), int(0.05*n))
x0 = position(i_x0)
xf = position(i_xf)
print("x0: {}, xf: {}".format(x0, xf))

# Define cameras and building placement
ang_lim = [0*np.pi/4, 2*np.pi/4] # [rad]
start_ang = ang_lim[0] # [rad]
fov_ang = np.deg2rad(20) # [rad]
rot_vel = (ang_lim[1]-ang_lim[0])/n*1.5 # [rad/unit time]
fov_rng = 0.75 # [m]
endtime = int(ang_lim[1]*2/rot_vel) # [unit time]
t = np.linspace(0, endtime-1, endtime)

def fov_cam(start_ang, ang_lim, rot_vel, fov_ang, fov_rng, t):
    # Compute Camera angle (centerline of FOV) at each timestamp t
    ang_prev = start_ang
    fov_ang_vec = np.zeros(len(t))
    dt = (t[1]-t[0])
    # Iteration until it hits the angle limit
    iter_lim = ang_lim[1]/rot_vel
    for i in range(len(t)-1):
        if i < iter_lim:
            fov_ang_curr = ang_prev + rot_vel*dt
        # Camera rotates back once reached its rotation limit
        else:
            fov_ang_curr = ang_prev - rot_vel*dt
        ang_prev = fov_ang_curr
        fov_ang_vec[i+1] = fov_ang_curr
    return fov_ang_vec

def position_cam(n_cam):
    camposx = np.zeros(n_cam)
    camposy = np.zeros(n_cam)
    for i in range(n_cam):
        camx = (i+1)/(n_cam+1)
        camy = 0
        camposx[i] = camx
        camposy[i] = camy
    return  np.array([camposx, camposy]).transpose()

n_cam = 1
campos = position_cam(n_cam)
camvec = fov_cam(start_ang, ang_lim, rot_vel, fov_ang, fov_rng, t)
print("campos: {}".format(campos))
print("camvec length: {}".format(len(camvec)))

# Define Initial, Final position of the Camera FOV (centerline)
end_in = int(len(t)/2)#random.randint(0, len(t)-1)
end_ang = camvec[end_in] # Arbirary number for now
print("end_index: {}, FOV angle at final point[deg]: {}".format(end_in, np.rad2deg(camvec[end_in])))

x0: [0.2 0.9], xf: [0.9  0.05]
campos: [[0.5 0. ]]
camvec length: 28
end_index: 14, FOV angle at final point[deg]: 90.0


In [6]:
# Compute Cost Function
def costfunc(x0, x1):
    # How do we add restriction from camera FOV?
    s = np.sqrt((x0[0]-x1[0])**2 + (x0[1]-x1[1])**2)
    return s

In [5]:
import copy

# Initialize Final Point
i_x_prev = i_xf
i_theta_prev = end_ang
i_theta = i_theta_prev
i_x = i_x_prev
V = np.zeros((n, n))
live = [[i_x[0], i_x[1]]]
live_hist = {}
live_hist['0'] = live

commentTrigger = False

# Bound within map
def bound(i_x):
    return (i_x[0] >= 0 and i_x[0]<n and i_x[1] >= 0 and i_x[1]<n)

# Bound outside of camera FOV
def bound_cam(i_x, campos, fov_ang_curr, fov_ang, fov_rng):
    # Convert i_x to position
    i_xPos = position(i_x)
    # distance to node
    dist = np.sqrt((campos[0]-i_xPos[0])**2 + (campos[1]-i_xPos[1])**2)
    # Angle to node
    ang = np.arctan2(i_xPos[1]-campos[1], i_xPos[0]-campos[0])
    return (ang >= fov_ang_curr-fov_ang/2 and ang < fov_ang_curr+fov_ang/2 and dist >= 0 and dist < fov_rng)

# Possible Actions
movement = [
    (1, 0), (-1, 0),  # Horizontal
    (0, 1), (0, -1),  # Vertical
    (0, 0)           # Stationary
    #(1, 1), (1, -1), (-1, 1), (-1, -1) # Diagonal
]

# Compute moves from current node
V_data = []
counter = 0
kek = end_in
while len(live) > 0:
    fig = plt.figure()
    new = set()
    end_in_prev = kek
    if end_in_prev < 0:
        end_in_prev = end_in_prev + len(camvec)
    print("end_in_prev: {}".format(end_in_prev))
    for p in live:
        if commentTrigger:
            print("counter: {}, p: {}".format(counter, p))
            print("counter: {}, Position(p): {}".format(counter, position(p)))
        for j in range(len(campos)):
            if not bound_cam(p, campos[j], camvec[end_in_prev], fov_ang, fov_rng):
                for a in movement:
                    pa = (p[0] - a[0], p[1] - a[1])
                    if bound(pa) and not bound_cam(pa, campos[j], camvec[end_in_prev], fov_ang, fov_rng):
                        if commentTrigger:
                            print("counter: {}, Position(p): {}".format(counter, position(p)))
                            print("bound_cam: {}, Position(pa): {}".format(bound_cam(pa, campos[j], camvec[end_in_prev], fov_ang, fov_rng), position(pa)))
                        V_new = costfunc(p, pa) + V[p[0], p[1]]
                        V_old = V[pa[0], pa[1]]
                        if V_old == 0 or V_new < V_old:
                            V[pa[0], pa[1]] = V_new
                            new.add((pa[0], pa[1]))
                            temp = position(pa)
                            plt.plot(temp[0], temp[1], 'xb')
    live = new
    key = str(counter)
    live_hist[key] = live
    V_data.append(copy.copy(V))
    kek = end_in_prev-1
    counter += 1
    
    # ===============================
    # Visualization Purpose
    plt.pcolor(X, Y, V_data[-1].T, vmin=V_data[-1].min(), vmax=V_data[-1].max())
    for ind in range(n_cam):
        # Plot Camera Position
        plt.plot(campos[ind,0], campos[ind,1], 'or')

        # Plot Camera FOV
        k = end_in_prev
        theta = camvec[k]
        xend = campos[ind,0]+fov_rng*np.cos(theta)
        yend = campos[ind,1]+fov_rng*np.sin(theta)
        plt.plot([campos[ind,0], xend], [campos[ind,1], yend], '--r')

        # FOV LOS
        h = fov_rng*np.tan(fov_ang/2)
        Rprime = np.sqrt(h**2 + fov_rng**2)

        xprime1 = [campos[ind,0], campos[ind,0]+Rprime*np.cos(camvec[k]+fov_ang/2)]
        yprime1 = [campos[ind,1], campos[ind,1]+Rprime*np.sin(camvec[k]+fov_ang/2)]
        xprime2 = [campos[ind,0], campos[ind,0]+Rprime*np.cos(camvec[k]-fov_ang/2)]
        yprime2 = [campos[ind,1], campos[ind,1]+Rprime*np.sin(camvec[k]-fov_ang/2)]
        plt.plot(xprime1, yprime1, '-r')
        plt.plot(xprime2, yprime2, '-r')
    
    # Plot x0, xf
    plt.plot(x0[0], x0[1], 'or')
    plt.plot(xf[0], xf[1], 'ob')

    # Remaining Plotting
    plt.grid()
    tol = 0.1
    plt.xlim(-tol, 1+tol)
    plt.ylim(-tol, 1+tol)
    #plt.show()
    plt.close()
    
print("Length V_data: {}".format(len(V_data)))

end_in_prev: 14
end_in_prev: 13
end_in_prev: 12
end_in_prev: 11
end_in_prev: 10
end_in_prev: 9
end_in_prev: 8
end_in_prev: 7
end_in_prev: 6
end_in_prev: 5
end_in_prev: 4
end_in_prev: 3
end_in_prev: 2
end_in_prev: 1
end_in_prev: 0
end_in_prev: -1
end_in_prev: -2
end_in_prev: -3
end_in_prev: -4
end_in_prev: -5
end_in_prev: -6
end_in_prev: -7
end_in_prev: -8
end_in_prev: -9
end_in_prev: -10
end_in_prev: -11
end_in_prev: -12
end_in_prev: -13
end_in_prev: -14
end_in_prev: -15
end_in_prev: -16
end_in_prev: -17
end_in_prev: -18
end_in_prev: -19
end_in_prev: -20
end_in_prev: -21
end_in_prev: -22
end_in_prev: -23
end_in_prev: -24
end_in_prev: -25
end_in_prev: -26
end_in_prev: -27
end_in_prev: -28
end_in_prev: -29


IndexError: index -29 is out of bounds for axis 0 with size 28

<Figure size 640x480 with 0 Axes>

In [ ]:
import matplotlib.animation

# video animation
from tempfile import NamedTemporaryFile
from IPython.display import HTML

In [ ]:
def anim_dyn_prog(V_data):
    nV = len(V_data)
    fig = plt.figure(figsize=(10, 10))
    plt.pcolor(X, Y, V_data[0].T, vmin=V_data[-1].min(), vmax=V_data[-1].max())   
    
    def update(i):
        fig.clear()
        if i < nV:
            # Grid Coloring
            plt.pcolor(X, Y, V_data[i].T, vmin=V_data[-1].min(), vmax=V_data[-1].max())
            
            # Node Population
            # Find current index from dictionary and plot populated node
            curr_key = str(i)
            liveNode = live_hist[curr_key]
            primaris = 0
            for p in liveNode:
                cawl = position(p)
                plt.plot(cawl[0], cawl[1], 'xb', label='Live Node' if primaris == 0 else "")
                primaris += 1
            
            # Camera Visualization
            for ind in range(n_cam):
                # Plot Camera Position
                plt.plot(campos[ind,0], campos[ind,1], 'ow', label = 'Camera Position')

                # Plot Camera FOV
                k = end_in-i%len(t)
                theta = camvec[k]
                xend = campos[ind,0]+fov_rng*np.cos(theta)
                yend = campos[ind,1]+fov_rng*np.sin(theta)
                #plt.plot([campos[ind,0], xend], [campos[ind,1], yend], '--w')

                # FOV LOS
                h = fov_rng*np.tan(fov_ang/2)
                Rprime = np.sqrt(h**2 + fov_rng**2)

                xprime1 = [campos[ind,0], campos[ind,0]+Rprime*np.cos(camvec[k]+fov_ang/2)]
                yprime1 = [campos[ind,1], campos[ind,1]+Rprime*np.sin(camvec[k]+fov_ang/2)]
                xprime2 = [campos[ind,0], campos[ind,0]+Rprime*np.cos(camvec[k]-fov_ang/2)]
                yprime2 = [campos[ind,1], campos[ind,1]+Rprime*np.sin(camvec[k]-fov_ang/2)]
                plt.plot(xprime1, yprime1, '-w')
                plt.plot(xprime2, yprime2, '-w', label = 'Camera FOV' if ind == 0 else "")        
        # Plot x0, xf
        plt.plot(x0[0], x0[1], 'or', label = 'Initial Point')
        plt.plot(xf[0], xf[1], 'ob', label = 'Final Point')

        # Remaining Plotting
        plt.grid()
        tol = 0.01
        plt.xlim(-tol, 1+tol)
        plt.ylim(-tol, 1+tol)
        plt.legend(loc="upper right")

    return matplotlib.animation.FuncAnimation(fig, update, frames=nV, interval=100)

# Save Animation into MP4
anim = anim_dyn_prog(V_data)
anim.save('Vdata_Propagation_Obs.mp4', fps=5)

In [ ]:
# Sanity Check
SanityCheck = type(None)

# Comment Trigger
commentTrigger = False

# Compute Optimal Path
p = i_x0
p_hist_opt = [p]
count = 0
while p != i_xf:
    # Find current time to check where the camera angle is:
    curr_time = count%len(t)
    if commentTrigger:
        print('============')
        print("counter: {}, curr_time: {}".format(counter, curr_time))
    pa_opt = None
    V_old = V[p[0], p[1]]
    for a in movement:
        pa = (p[0] + a[0], p[1] + a[1])
        if commentTrigger:
            print("position pa: {}".format(position(pa)))
        for j in range(len(campos)):
            if commentTrigger:
                print(not bound_cam(pa, campos[j], camvec[curr_time], fov_ang, fov_rng))
            if bound(pa) and not bound_cam(pa, campos[j], camvec[curr_time], fov_ang, fov_rng):
                V_new = V[pa[0], pa[1]]
                if pa_opt is None or pa == i_xf or (V_new < V_opt and V_new < V_old):
                    V_opt = V_new
                    pa_opt = pa
            if pa == i_xf: break
    if count > 1000: break
    p = pa_opt
    # Check if p is empty
    # If p is empty: There is no solution exist s.t. x0 and xf can be connected without detection
    if isinstance(p, SanityCheck):
        break
    else:
        p_hist_opt.append(p)
        count += 1
pos_hist_opt = np.array([position(p) for p in p_hist_opt])
print(len(pos_hist_opt))
#print(pos_hist_opt)

In [ ]:
def anim_dyn_prog(V_data):
    nV = len(V_data)
    nP = len(pos_hist_opt)
    fig = plt.figure(figsize=(10, 10))
    plt.pcolor(X, Y, V_data[0].T, vmin=V_data[-1].min(), vmax=V_data[-1].max())
    
    def update(i):
        fig.clear()
#        if i < nV:
#            plt.pcolor(X, Y, V_data[i].T, vmin=V_data[-1].min(), vmax=V_data[-1].max())
#            # Camera Visualization
#            for ind in range(n_cam):
#                # Plot Camera Position
#                plt.plot(campos[ind,0], campos[ind,1], 'or')

#                # Plot Camera FOV
#                k = i%len(t)
#                theta = camvec[k]
#                xend = campos[ind,0]+fov_rng*np.cos(theta)
#                yend = campos[ind,1]+fov_rng*np.sin(theta)
#                plt.plot([campos[ind,0], xend], [campos[ind,1], yend], '--r')

#                # FOV LOS
#                h = fov_rng*np.tan(fov_ang/2)
#                Rprime = np.sqrt(h**2 + fov_rng**2)

#                xprime1 = [campos[ind,0], campos[ind,0]+Rprime*np.cos(camvec[k]+fov_ang/2)]
#                yprime1 = [campos[ind,1], campos[ind,1]+Rprime*np.sin(camvec[k]+fov_ang/2)]
#                xprime2 = [campos[ind,0], campos[ind,0]+Rprime*np.cos(camvec[k]-fov_ang/2)]
#                yprime2 = [campos[ind,1], campos[ind,1]+Rprime*np.sin(camvec[k]-fov_ang/2)]
#                plt.plot(xprime1, yprime1, '-r')
#                plt.plot(xprime2, yprime2, '-r')
        #else:
        if i < nP:
            # Plot from t0
            plt.pcolor(X, Y, V_data[nV-1].T, vmin=V_data[-1].min(), vmax=V_data[-1].max())

            # Plot Time Varying FOV
            plt.plot(pos_hist_opt[:i,0], pos_hist_opt[:i,1], '--k', label = 'Aggressor Position History')
            plt.plot(pos_hist_opt[i-1:i+1,0], pos_hist_opt[i-1:i+1,1], '-k')
            plt.plot(pos_hist_opt[i,0], pos_hist_opt[i,1], 'ok', label = 'Aggressor')
            for ind in range(n_cam):
                # Plot Camera Position
                plt.plot(campos[ind,0], campos[ind,1], 'ow', label = 'Camera Position' if ind == 0 else "")

                # Plot Camera FOV
                k = end_in - nP + i + 1
                #print('i: {}, k: {}, end_in: {}'.format(i, k, end_in))
                theta = camvec[k]
                xend = campos[ind,0]+fov_rng*np.cos(theta)
                yend = campos[ind,1]+fov_rng*np.sin(theta)
                #plt.plot([campos[ind,0], xend], [campos[ind,1], yend], '--r')

                # FOV LOS
                h = fov_rng*np.tan(fov_ang/2)
                Rprime = np.sqrt(h**2 + fov_rng**2)

                xprime1 = [campos[ind,0], campos[ind,0]+Rprime*np.cos(camvec[k]+fov_ang/2)]
                yprime1 = [campos[ind,1], campos[ind,1]+Rprime*np.sin(camvec[k]+fov_ang/2)]
                xprime2 = [campos[ind,0], campos[ind,0]+Rprime*np.cos(camvec[k]-fov_ang/2)]
                yprime2 = [campos[ind,1], campos[ind,1]+Rprime*np.sin(camvec[k]-fov_ang/2)]
                plt.plot(xprime1, yprime1, '-w')
                plt.plot(xprime2, yprime2, '-w', label = 'Camera FOV' if ind == 0 else "")
            
        # Timer Indication
        plt.title('Timestep: {}'.format(i))

        # Plot x0, xf
        plt.plot(x0[0], x0[1], 'or', label = 'Initial Point')
        plt.plot(xf[0], xf[1], 'ob', label = 'Final Point')

        # Remaining Plotting
        plt.grid()
        tol = 0.01
        plt.xlim(-tol, 1+tol)
        plt.ylim(-tol, 1+tol)
        plt.legend(loc="upper right")

    return matplotlib.animation.FuncAnimation(fig, update, frames=nP, interval=50, repeat=False)

In [ ]:
anim = anim_dyn_prog(V_data)
anim.save('Aggressor_Insertion_Obs.mp4', fps=2.5)